#**Libraries / path definition / various functions**

In [ ]:
!pip install torchmetrics

In [ ]:
import cv2
import numpy as np
import math
import sys
import os
from skimage.filters import threshold_multiotsu
import random
import matplotlib.pyplot as plt
import re
from PIL import Image
from tqdm import tqdm
from skimage import img_as_ubyte, io
import torch
import torch.nn.functional as F
import torchmetrics
from torchmetrics.classification import MulticlassJaccardIndex

In [ ]:
# Define the project directory path
project_dir = '/content/gdrive/MyDrive/'

# Define the name of the folder containining the datasets. All data are in tiff format (or equivalent).
# The expected data directory structure is as follows:
# Datasets
# |_Sample1
# |  |_img
# |     |_image1.tiff
# |     |_image2.tiff
# |     |_...
# |  |_mask
# |     |_mask1.tiff
# |     |_mask2.tiff
# |     |_...
# |  |_...
# ...
dataset_name = "some_dataset"
sample_name = "sample3"
num_classes = 3
crop_size = (560, 560)

######

# Add the DinoV2 code directory to the system path for module imports
sys.path.append(os.path.join(project_dir, "code/DinoV2/"))

# Define the data directory path within the project directory
data_directory = os.path.join(project_dir, 'data')

# Define the input directory path to the dataset for segmentation
input_directory = os.path.join(data_directory, dataset_name, sample_name)

# **Various functions**


In [ ]:
# Define the Non-Local Means filter function
def non_local_means_filter(image, h=10, templateWindowSize=7, searchWindowSize=21):
    return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)

# Define the Otsu multi-thresholding function
def otsu_multi(image, num_classes):
    thresholds = threshold_multiotsu(image, classes=num_classes)
    regions = np.digitize(image, bins=thresholds)
    return regions

# Define a function to center crop the image
def center_crop(image, crop_size):
    height, width = image.shape[:2]
    crop_height, crop_width = crop_size
    if height < crop_height or width < crop_width:
        raise ValueError("Crop size must be smaller than the image size")
    top = (height - crop_height) // 2
    left = (width - crop_width) // 2
    cropped_image = image[top:top + crop_height, left:left + crop_width]
    return cropped_image

def remap_values(array, unique_values, target_values):
    mapping_dict = {old_val: new_val for old_val, new_val in zip(unique_values, target_values)}
    remapped_array = np.copy(array)
    for old_val, new_val in mapping_dict.items():
        remapped_array[array == old_val] = new_val
    return remapped_array

def mapping(target, pred, num_classes):
    unique_values_target = np.unique(target)
    unique_values_pred = np.unique(pred)

    # Desired unique values after remapping
    target_values = np.arange(num_classes)

    if not np.array_equal(unique_values_target, target_values):
        target = remap_values(target, unique_values_target, target_values)

    if not np.array_equal(unique_values_pred, target_values):
        pred = remap_values(pred, unique_values_pred, target_values)

    return target, pred


def process_images(input_directory, crop_size, num_classes):
    image_files = [f for f in os.listdir(os.path.join(input_directory, 'images')) if os.path.isfile(os.path.join(input_directory, 'images', f))]
    mask_files = [f for f in os.listdir(os.path.join(input_directory, 'masks')) if os.path.isfile(os.path.join(input_directory, 'masks', f))]
    jaccard = torchmetrics.classification.MulticlassJaccardIndex(num_classes=num_classes)

    def extract_number(file_name):
        match = re.search(r'\d{4}', file_name)
        return int(match.group()) if match else 0

    image_files = sorted(image_files, key=extract_number)
    mask_files = sorted(mask_files, key=extract_number)
    combined = list(zip(image_files, mask_files))

    results = []
    list_iou = []
    total_iou = 0
    count = 0

    for img_file, mask_file in tqdm(combined, desc="Loading images and masks", total=len(image_files)):
        img_path = os.path.join(input_directory, 'images', img_file)
        mask_path = os.path.join(input_directory, 'masks', mask_file)

        img = io.imread(img_path, as_gray=True)
        mask = io.imread(mask_path, as_gray=True)

        filtered_image = non_local_means_filter(img, h=15, templateWindowSize=7, searchWindowSize=21)

        otsu_segmented = otsu_multi(filtered_image, num_classes)

        cropped_scanner = center_crop(img, crop_size)
        cropped_pred = center_crop(otsu_segmented, crop_size)
        cropped_mask = center_crop(mask, crop_size)

        cropped_mask, cropped_pred = mapping(cropped_mask, cropped_pred, num_classes)

        # Convert to tensors with uint8 data type
        ground_truth_mask_tensor = torch.tensor(cropped_mask, dtype=torch.uint8)
        otsu_mask_tensor = torch.tensor(cropped_pred, dtype=torch.uint8)

        # Add batch dimension
        ground_truth_mask_tensor = ground_truth_mask_tensor.unsqueeze(0)  # Shape: [1, height, width]
        otsu_mask_tensor = otsu_mask_tensor.unsqueeze(0)  # Shape: [1, height, width]

        try:
            mIoU = jaccard(otsu_mask_tensor, ground_truth_mask_tensor).item()
        except Exception as e:
            print(f"Error calculating mIoU: {e}")
            continue

        list_iou.append(mIoU)

        total_iou += mIoU
        count += 1

        results.append((cropped_scanner, cropped_mask, cropped_pred))

    if count > 0:
        average_iou = total_iou / count
    else:
        average_iou = 0
        print("No valid image-mask pairs processed.")

    return average_iou, results, list_iou

def display_random_set(image_mask_pred_list):
    random_index = random.randint(0, len(image_mask_pred_list) - 1)
    random_set = image_mask_pred_list[random_index]
    img, ground_truth_mask, pred_mask = random_set
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Image' + str(random_index))
    ax[1].imshow(ground_truth_mask, cmap='gray')
    ax[1].set_title('Ground Truth Mask')
    ax[2].imshow(pred_mask, cmap='gray')
    ax[2].set_title('Otsu segmented')
    plt.show()

# **Segmentation**

In [ ]:
average_iou, results, iou = process_images(input_directory, crop_size, num_classes)
print("Average IoU:", average_iou)

# **Display**

In [ ]:
display_random_set(results)